# Проект - Рекомендация тарифов


# Задача проекта
- построить модель для задачи классификации
- с максимально большим значением accuracy > 0.75

# Описание проекта
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.


# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 
Известно:
- calls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


# Инструкция по выполнению проекта
1.	Откройте файл с данными и изучите его. Путь к файлу: /datasets/users_behavior.csv. 
2.	Разделите исходные данные на обучающую, валидационную и тестовую выборки.
3.	Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.
4.	Проверьте качество модели на тестовой выборке.
5.	Дополнительное задание: проверьте модели на вменяемость. 

In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

In [2]:
# для оформления проекта использовал модуль tqdm для индикации выполнения, информация по модулю ниже
from tqdm import trange 
from tqdm.notebook import trange

Модуль tqdm предназначен для быстрого и расширяемого внедрения индикаторов выполнения (progressbar) во внешние интерфейсы программ на Python, предоставляя конечным пользователям визуальную индикацию хода вычислений или передачи данных. Он также будет полезен в целях отладки, как в качестве инструмента профилирования, так и в качестве способа отображения информации журнала итеративной задачи. Благодаря простоте использования библиотека также является идеальным кандидатом для включения в образовательные курсы Python.

https://docs-python.ru/packages/tqdm-progressbar/

## Откройте и изучите файл

In [3]:
# загрузка файла и вывод первых 5 строк df
data = pd.read_csv('users_behavior.csv')
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
# вывод информации по df
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
# проверим на явные дупликаты
data.duplicated().sum()

0

In [6]:
# переведем тип столбцов 'calls', 'messages' в тип integer
for column in data[['calls', 'messages']]:
    data[column] = data[column].astype('int')

In [7]:
# сравним использование услуг по тарифам «Ультра» — 1, «Смарт» — 0
data.pivot_table(index='is_ultra', values=['calls', 'minutes', 'messages', 'mb_used'], aggfunc='mean').reset_index()

,is_ultra,calls,mb_used,messages,minutes
0,0,58.463437,16208.466949,33.384029,405.942952
1,1,73.392893,19468.823228,49.363452,511.224569


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int32  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int32  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int32(2), int64(1)
memory usage: 100.6 KB


In [9]:
data.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


### Выводы, итоги, информации п.1
- пропусков, дубликатов нет, данные подготовлены
- сделана замена типа столбцов 'calls', 'messages' на тип integer

### Задача
- построить модель для задачи классификации, которая выберет подходящий тариф
- с максимально большим значением accuracy > 0.75

## Разбейте данные на выборки

In [10]:
# разделим данные на признаки и целевой признак
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

In [11]:
# разделим на обучающую и тестовую выборки в пропорции 4 к 1
train_features, test_features, train_target, test_target = train_test_split(features, target, 
                                                                            test_size=0.20,random_state=12345)

In [12]:
# разделим обучающую выборку на обучающую и валидационную выборкив пропорции 3 к 1
train_features, valid_features, train_target, valid_target = train_test_split(train_features, train_target, 
                                                                              test_size=0.25, random_state=12345)

In [13]:
# данные о размере выборок
print(train_features.shape, valid_features.shape, test_features.shape) 

(1928, 4) (643, 4) (643, 4)


### Выводы, итоги, информации п.2
- данные разделены на признаки и целевой признак
- данные разбиты на обучающую, валидационную и тестовую выборки в пропорции 3:1:1 и подготовлены к обучению

### Задача
- построить модель для задачи классификации, которая выберет подходящий тариф
- с максимально большим значением accuracy > 0.75

## Построение моделей для классификации

In [14]:
# обучение модели на алгоритме DecisionTreeClassifier с перебором параметра максимальная Глубина

best_model_DTC = None
best_result_DTC = 0

for depth in trange(1, 11):
    model_DTC = DecisionTreeClassifier(random_state=12345, max_depth=depth).fit(train_features, train_target) 
    predictions_DTC = model_DTC.predict(valid_features) 
    result_DTC = accuracy_score(valid_target, predictions_DTC)
    if result_DTC > best_result_DTC:
        best_result_DTC = result_DTC
        best_model_DTC = model_DTC
        
      
print("Accuracy лучшей модели:", round(best_result_DTC,5))
print('Лучшая модель:', best_model_DTC)


Accuracy лучшей модели: 0.77449
Лучшая модель: DecisionTreeClassifier(max_depth=7, random_state=12345)


In [15]:
# обучение модели на алгоритме RandomForestClassifier с перебором Гиперпараметра кол-во Деревьев

best_model_RFC = None
best_result_RFC = 0

for est in trange(1, 11):
    model_RFC = RandomForestClassifier(random_state=12345, n_estimators=est).fit(train_features, train_target) 
    predictions_RFC = model_RFC.predict(valid_features) 
    result_RFC = accuracy_score(valid_target, predictions_RFC)
    if result_RFC > best_result_RFC:
        best_result_RFC = result_RFC
        best_model_RFC = model_RFC
        
      
print("Accuracy лучшей модели:", round(best_result_RFC,5))
print('Лучшая модель:', best_model_RFC)


Accuracy лучшей модели: 0.78849
Лучшая модель: RandomForestClassifier(n_estimators=10, random_state=12345)


In [16]:
# обучение модели на алгоритме RandomForestClassifier с перебором Гиперпараметров: кол-во Деревьев, максим. Глубина

best_model_RFC_2 = None
best_result_RFC_2 = 0

for est in trange(10, 51, 10):
    for depth in range (1, 11):
        model_RFC_2 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_RFC_2.fit(train_features, train_target) 
        predictions_RFC_2 = model_RFC_2.predict(valid_features) 
        result_RFC_2 = accuracy_score(valid_target, predictions_RFC_2)
        if result_RFC_2 > best_result_RFC_2:
            best_model_RFC_2 = model_RFC_2
            best_result_RFC_2 = result_RFC_2

            
print("Accuracy лучшей модели:", round(best_result_RFC_2,5))
print('Лучшая модель:', best_model_RFC_2)    


Accuracy лучшей модели: 0.79782
Лучшая модель: RandomForestClassifier(max_depth=10, n_estimators=50, random_state=12345)


In [17]:
# обучение модели на алгоритме LogisticRegression

model_LR = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000).fit(train_features, train_target)
model_LR.score(valid_features, valid_target)

    
print("Accuracy модели:", round(model_LR.score(valid_features, valid_target),5))
print('Модель:', model_LR)

Accuracy модели: 0.72628
Модель: LogisticRegression(max_iter=1000, random_state=12345)


### Выводы, итоги, информации п.3
- DecisionTreeClassifier: Accuracy лучшей модели: 0.77449, Лучшая модель:  DecisionTreeClassifier(max_depth=7)
- RandomForestClassifier: Accuracy лучшей модели: 0.78849, Лучшая модель: RandomForestClassifier(n_estimators=10)
- RFC_2: Accuracy лучшей модели: 0.79782, Лучшая модель: RandomForestClassifier(max_depth=10, n_estimators=50)
- LogisticRegression: Accuracy модели: 0.72628, Модель: LogisticRegression(max_iter=1000)


- все модели, кроме Логистической регресси, показывают Accuracy > 0.75
- лучшая модель по Accuracy RandomForestClassifier(max_depth=10, n_estimators=50), Accuracy=0.79782

## Проверка модели на тестовой выборке

In [18]:
# проверка на тестовой выборке модели на основе алгоритма DecisionTreeClassifier 
round(best_model_DTC.score(test_features, test_target),5)

0.78849

In [19]:
# проверка на тестовой выборке модели на основе алгоритма RandomForestClassifier 
round(best_model_RFC.score(test_features, test_target),5)

0.78694

In [20]:
# проверка на тестовой выборке модели на основе алгоритма RandomForestClassifier_2
round(best_model_RFC_2.score(test_features, test_target),5)

0.79938

In [21]:
# проверка на тестовой выборке модели на основе алгоритма LogisticRegression
round(model_LR.score(test_features, test_target),5)

0.75894

### Выводы, итоги, информации п.4

лучшая модель RFC_2
- RandomForestClassifier(max_depth=10, n_estimators=50, random_state=12345)
- Accuracy на валидации: 0.79782
- Accuracy на тесте: 0.79937


## Проверка модели на адекватность
- посмотрим на наиболе часто встречащийся класс в наших данных
- сравним его долю с оценкой Accurasy на выбранной модели

In [22]:
data.groupby('is_ultra').count()

,calls,minutes,messages,mb_used
is_ultra,,,,
0,2229,2229,2229,2229
1,985,985,985,985


Наиболее часто встречающийся класс в наших данных это is_ultra == 0. Количество значений = 2229. Всего же значений в df = 3214. Значит самый часто встречающийся класс "занимает" 69% (2229 / 3214 == 0.6935)

Оценка Accuracy нашей лучшей модели на тесте: 0.79937

Вывод - наша модель RFC_2 адекватна.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


## Выводы, итоги, информация по всему проекту

- загруженны и изучены данные: данные чистые, пропусков и дубликатов нет
- данные разделены на обучающую, валидационную и тестовую выборки, подготвлены к обучению
- проведенно исследование по четырем моделям: 
    - Decision Tree Classifier
    - RandomForestClassifier с изменением Гиперпараметра кол-во Деревьев
    - RandomForestClassifier с изменением Гиперпараметров: кол-во Деревьев и максимальная Глубина
    - Logistic Regression
- все модели, кроме Логистической регресси, показывают на валидационно выборке Accuracy > 0.75
- модели проверил на тестовой выборке: все модели показывают Accuracy > 0.75
- лучшая модель по Accuracy - RandomForestClassifier(max_depth=10, n_estimators=50 random_state=1234), Accuracy=0.79782
- сделали оценку на адекватность модели